# SMT Solver for VLSI

In [ ]:
from itertools import combinations
from z3 import *
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import time

In [ ]:
def plot_result(dims, w, ys, xs):
    lims = (0, max(w, max([ys[i]+dims[i][1] for i in range(len(ys))])))

    fig1 = plt.figure(figsize=(10, 10))
    ax1 = fig1.add_subplot(111, aspect='equal')
    for i in range(len(xs)):
        ax1.add_patch(patches.Rectangle((xs[i], ys[i]), dims[i][0], dims[i][1], edgecolor='black', facecolor=random.choice(['r', 'g', 'y'])))

    plt.xticks(range(w+1))
    plt.yticks(range(max([ys[i]+dims[i][1] for i in range(len(ys))])+1))
    plt.show()

In [ ]:
def max_z3(vars):
    max = vars[0]
    for v in vars[1:]:
        max = If(v > max, v, max)
    return max

In [ ]:
def read_instance(instance_id):
    filepath = "instances/ins-" + str(instance_id) + ".txt"
    with open(filepath, "r") as f_in:
        f = f_in.readlines()
        for i in range(len(f)):
            if not f[i][-1].isnumeric():
                f[i] = f[i][:-1]
                
        W = int(f[0])
        n = int(f[1])
        dims_line = f[2].split(" ")    
        dims = [[int(dims_line[0]), int(dims_line[1])]]
        for i in range(1, int(f[1])-1):
            dims_line = f[2 + i].split(" ")
            dims.append([int(dims_line[0]), int(dims_line[1])])
        dims_line = f[-1].split(" ")
        dims.append([int(dims_line[0]), int(dims_line[1])])
        
        dims = sorted(dims, key=lambda l:l[0]*l[1], reverse=True)
        
    return dims, W, n

In [ ]:
def cumulative(opt, s, d, r, c, time):
    for t in range(time):
        tmp_sum = 0
        for i in range(len(s)):
            tmp_sum += If(And(s[i] <= t, t < s[i] + d[i]), 1, 0) * r[i]
        opt.add(c >= tmp_sum) 
    return opt

In [ ]:
for instance_m in range(1, 20): 
    dims, W, n = read_instance(instance_m)
    # Solver
    opt = Optimize()

    sol = [[Int("x_%s" % (i)), Int("y_%s" % (i))] for i in range(n)]

    # overlapping
    for i in range(n):
        for j in range(n):
            if i != j:
                opt.add(Or(sol[i][0] + dims[i][0] <= sol[j][0],
                           sol[j][0] + dims[j][0] <= sol[i][0],
                           sol[i][1] + dims[i][1] <= sol[j][1],
                           sol[j][1] + dims[j][1] <= sol[i][1]))

    # width
    for i in range(n):
        opt.add(sol[i][0] + dims[i][0] <= W)
        opt.add(sol[i][0] >= 0) # min
        opt.add(sol[i][0] <= W - min([dims[i][0] for i in range(n)])) # max

    # height min
    for i in range(n):
        opt.add(sol[i][1] >= 0)

    # Symmetry breaking
    for i in range(n):
        for j in range(n):
            if i<j and dims[i][0] == dims[j][0] and dims[i][1] == dims[j][1]:
                opt.add(Or(sol[i][0]<=sol[j][0], sol[i][1]<=sol[j][1]))

    for i in range(n):
        for j in range(n):
            if i<j and dims[i][0] == dims[j][0]:
                opt.add(Implies(sol[i][0]==sol[j][0], sol[i][1]<=sol[j][1]))
            if i<j and dims[i][1] == dims[j][1]:
                opt.add(Implies(sol[i][1]==sol[j][1], sol[i][0]<=sol[j][0]))

    height_opt = Int('height_opt')

    # Biggest rectangle
    opt.add(And(sol[0][0] <= 1+(W-dims[0][0])/2, sol[0][1] <= 1+(height_opt-dims[0][1])/2))

    min_height = int(sum([dims[i][1]*dims[i][0] for i in range(n)])/W)
    max_height = 2*max(max([dims[i][1] for i in range(n)]), min_height)
    
    # Cumulative on x
    opt = cumulative(opt, [sol[i][0] for i in range(n)], [dims[i][0] for i in range(n)], 
                    [dims[i][1] for i in range(n)], height_opt, W)
    
    # Cumulative on y
    opt = cumulative(opt, [sol[i][1] for i in range(n)], [dims[i][1] for i in range(n)], 
                    [dims[i][0] for i in range(n)], W, max_height)

    opt.add(height_opt <= max_height) # upper bound
    opt.add(height_opt >= min_height) # lower bound
    objective = height_opt == max_z3([sol[i][1] + dims[i][1] for i in range(n)])
    opt.add(objective)
    
    for i in range(n):
        opt.add(sol[i][1] <= height_opt-dims[i][1])
    opt.minimize(height_opt)
    
    opt.set("timeout", 300*1000)  # 5 minutes timeout

    start_time = time.time()
    opt.check()
    m = opt.model()
    print("Solving time (s): ", time.time()-start_time)

    ys = []
    xs = []
    for i in range(n):
        xs.append(int(m.evaluate(sol[i][0]).as_string()))
        ys.append(int(m.evaluate(sol[i][1]).as_string()))

    f = open("out/out-" + str(instance_m) + ".txt", "w")
    f.write(str(W) + " " + m.evaluate(height_opt).as_string() + "\n")
    f.write(str(n) + "\n")
    for i in range(n):
        if i < n-1:
            f.write(str(dims[i][0]) + " " + str(dims[i][1]) + " " + m.evaluate(sol[i][0]).as_string() + " " + m.evaluate(sol[i][1]).as_string() + "\n")
        else:
            f.write(str(dims[i][0]) + " " + str(dims[i][1]) + " " + m.evaluate(sol[i][0]).as_string() + " " + m.evaluate(sol[i][1]).as_string())
    f.close()
    
    #plot_result(dims, W, ys, xs)